# Saving PyBaMM models to file

Models which are discretised (i.e. ready to solve/ previously solved, see [this notebook](https://github.com/pybamm-team/PyBaMM/blob/develop/docs/source/examples/notebooks/spatial_methods/finite-volumes.ipynb) for more information on the pybamm.Discretisation class) can be serialised and saved to a JSON file, ready to be read in again either in PyBaMM, or a different modelling library. 

In the example below, we build and solve a basic DFN model, and then save the model out to `sim_model_example.json`, which should have appear in the 'models' directory.

In [ ]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm

# do the example
dfn_model = pybamm.lithium_ion.DFN()
dfn_sim = pybamm.Simulation(dfn_model)
dfn_sim.solve([0, 3600])

dfn_sim.save_model("sim_model_example")

This model file can then be read in and solved by choosing a solver, and running as below.

In [ ]:
# Recreate the pybamm model from the JSON file
new_dfn_model = pybamm.load_model("sim_model_example.json")

sim_reloaded = pybamm.Simulation(new_dfn_model)
sim_reloaded.solve([0, 3600])

It would be nice to plot the results of the two models, to confirm that they are producing the same result.

However, notice that running the code below generates an error stating that the model variables were not provided during the reading in of the model.

In [ ]:
dfn_models = [dfn_model, new_dfn_model]
sims = []
for model in dfn_models:
    plot_sim = pybamm.Simulation(model)
    plot_sim.solve([0, 3600])
    sims.append(plot_sim)

pybamm.dynamic_plot(sims, time_unit="seconds")

To be able to plot the results from a serialised model, the mesh and model variables need to be saved alongside the model itself.

To do this, set the `variables` option to `True` when saving the model as in the example below; notice how the models will now plot nicely.

In [ ]:
# using the first simulation, save a new file which includes a list of the model variables
dfn_sim.save_model("sim_model_variables", variables=True)

# read the model back in
model_with_vars = pybamm.load_model("sim_model_variables.json")

# plot the pre- and post-serialisation models together to prove they behave the same
models = [dfn_model, model_with_vars]
sims = []
for model in models:
    sim = pybamm.Simulation(model)
    sim.solve([0, 3600])
    sims.append(sim)

pybamm.dynamic_plot(sims, time_unit="seconds")

## Saving from Model

Alternatively, the model can be saved directly from the Model class.

Note that at the moment, only models derived from the BaseBatteryModel class can be serialised; those built from scratch using pybamm.BaseModel() are currently unsupported.

First set up the model, as explained in detail for the [SPM](https://github.com/pybamm-team/PyBaMM/blob/develop/docs/source/examples/notebooks/models/SPM.ipynb).

In [ ]:
# create the model
spm_model = pybamm.lithium_ion.SPM()

# set up and discretise ready to solve
geometry = spm_model.default_geometry
param = spm_model.default_parameter_values
param.process_model(spm_model)
param.process_geometry(geometry)
mesh = pybamm.Mesh(geometry, spm_model.default_submesh_types, spm_model.default_var_pts)
disc = pybamm.Discretisation(mesh, spm_model.default_spatial_methods)
disc.process_model(spm_model)

Then save the model. Note that in this case the model variables and the mesh must be provided directly.

In [6]:
# Serialise the spm_model, providing the varaibles and the mesh
spm_model.save_model("example_model", variables=spm_model.variables, mesh=mesh)

Now you can read the model back in, solve and plot.

In [ ]:
# read back in
new_spm_model = pybamm.load_model("example_model.json")

# select a solver and solve
new_spm_solver = new_spm_model.default_solver
new_spm_solution = new_spm_solver.solve(new_spm_model, [0, 3600])

# plot the solution
new_spm_solution.plot()

## Making edits to a serialised model

As mentioned at the begining of this notebook, only models which have already been discretised can be serialised and readh back in. This means that after serialisation, the model *cannot be edited*, as the non-discretised elements of the model such as the original rhs are not saved.

If you are likely to want to save a model and then edit it in the future, you may wish to use the `Simulation.save()` functionality to pickle your simulation, as described in [tutorial 6](https://github.com/pybamm-team/PyBaMM/blob/develop/docs/source/examples/notebooks/getting_started/tutorial-6-managing-simulation-outputs.ipynb).

Before finishing we will remove the data files we saved so that we leave the directory as we found it

In [3]:
import os
os.remove("example_model.json")
os.remove("sim_model_example.json")
os.remove("sim_model_variables.json")

## References

The relevant papers for this notebook are:

In [3]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/